# Extensive EDA and Modeling XGB Hyperopt

Extensive EDA and modeling XGB Hyperopt

we will predict fraud in transactions

## Library Import

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# Standard plotly imports
#import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import iplot, init_notebook_mode
#import cufflinks
#import cufflinks as cf
import plotly.figure_factory as ff

# Using plotly + cufflinks in offline mode
init_notebook_mode(connected=True)
#cufflinks.go_offline(connected=True)

# Preprocessing, modelling and evaluating
# from sklearn import preprocessing
# from sklearn.metrics import confusion_matrix, roc_auc_score
# from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
# from xgboost import XGBClassifier
# import xgboost as xgb

## Hyperopt modules
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK, STATUS_RUNNING
from functools import partial

import os
import gc

## EDA

### Questions
명목형 변수와 거래금액(Transaction Amounts)를 기반으로 탐색하기로 한다. 목표들은 아래와 같다.   
- 우리의 데이터들의 타입은 어떠한가?
- 얼마나 많은 행, 열이 결측치를 가지는가?
- 목표 분포는 무엇인가?
- 사기의 거래와 사기가 아닌 거래의 거래 값 분포는 어떠한가?
- 주요 사기 제품이 있는가?
- 어느 피쳐나 타겟이 흥미로운 패턴을 보이는가?
- 탐색을 통해 더 많은 새로 제기할 질문이 있는가?

### Training Data for EDA

 이 대회에서 우리는 Binary변수 isFraud처럼 온라인 거래가 사기일 개연성을 예측해야 한다.
데이터는 identity와 transaction으로 나뉘어져 있으며, TransactionID로 연결되어 있다.

##### Categorical Features - Transaction
- ProductCD
- card1 - card6
- addr1, addr2
- P_emaildomain
- R_emaildomain
- M1 - M9

##### Categorical Features - Identity
- DeviceType
- DeviceInfo
- id_12 - id_38

> TransactionDT 이것은 주어진 참조 datetime으로부터의 타임 델타 피쳐이다.(actual timestamp가 아님)

#### Data Import and Reduce Memory

In [ ]:
# identity data and trans data
df_id = pd.read_csv('../input/ieee-fraud-detection/train_identity.csv')
df_trans = pd.read_csv('../input/ieee-fraud-detection/train_transaction.csv')

#### Data Handle Function

In [ ]:
def resume_table(df):
    """make and show summary of missing,unique,1st,2nd,3rd columns of df"""
    """데이터의 결측치,유니크 값 등을 요약해서 표로 보여주는 함수"""
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes, columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name', 'dtypes']]
    summary['Missing'] = df.isnull().sum().values
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values

    for name in summary['Name'].value_counts().index:
        summary.loc[summary['Name'] == name, 'Entropy'] = round(stats.entropy(df[name].value_counts(normalize=True), base=2), 2)
    return summary

def reduce_mem_usage(df, verbose=True):
    """각 컬럼의 최대치, 최소치 범위를 탐색하여 딱 맞는 데이터 타입으로 변경한다. 메모리 사용을 최대한 줄이기 위함."""
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':        
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def calc_outliers(df_num):
    """Caclulate outliers"""
    """set Std * 3 as outlier cutline"""
    # calculating mean and std of the array
    data_mean, data_std = np.mean(df_num), np.std(df_num)
    
    # Set cutline by 68–95–99.7 rule(empirical rule)
    cut = data_std * 3

    # 평균값에 커트라인 값을 각각 감산, 가산한 값을 낮은 경계, 높은 경계로 각각 둔다.
    lower, upper = data_mean - cut, data_mean + cut

    # 낮은 이상치 리스트, 높은 이상치 리스트, 높낮 이상치 토탈 리스트를 각각 작성한다.
    # List of Low Outliers than lower, High Outliers than upper, 
    outliers_lower = [x for x in df_num if x < lower]
    outliers_higher = [x for x in df_num if x > upper]
    outliers_total = outliers_lower.copy() + outliers_higher.copy() #[x for x in df_num if x < lower or x > upper]

    # 이상치가 제거된 리스트를 작성한다.
    outliers_removed = [x for x in df_num if x > lower or x < upper]

    print('Identified lowest outliers: %d' % len(outliers_lower)) # printing total number of values in lower cut of outliers
    print('Identified upper outliers: %d' % len(outliers_higher)) # printing total number of values in higher cut of outliers
    print('Total outlier observations: %d' % len(outliers_total)) # printing total number of values outliers of both sides
    print('Non-outlier observations: %d' % len(outliers_removed)) # printing total number of non outlier values
    print('Total percentual of Outliers: ', round((len(outliers_total) / len(outliers_removed)) * 100, 4))

    return

In [ ]:
# Reduce Memories of Training Data
df_trans = reduce_mem_usage(df_trans)
df_id = reduce_mem_usage(df_id)

#### Watch Data summary

In [ ]:
# There are so many reatures in training data, so we make resume table for only 25 features.
resume_table(df_trans)[:25]

#### Target Value

##### Target value Distribution

In [ ]:
df_trans['TransactionAmt'] = df_trans['TransactionAmt'].astype(float)
total = len(df_trans)
total_amt = df_trans.groupby(['isFraud'])['TransactionAmt'].sum().sum()

plt.figure(figsize=(16, 6))
plt.subplot(121)
g = sns.countplot(x='isFraud', data=df_trans)
g.set_title('Fraud Transactions Distribution \n# 0: No Fraud | 1: Fraud #', fontsize=22)
g.set_xlabel('Is fraud?', fontsize=18)
g.set_ylabel('Count', fontsize=18)
for p in g.patches:
#     print(p)
    height = p.get_height()
    g.text(
        p.get_x() + p.get_width() / 2.,
        height + 3,
        '{:1.2f}%'.format(height/total*100),
        ha="center", fontsize=15)

perc_amt = (df_trans.groupby(['isFraud'])['TransactionAmt'].sum())
perc_amt = perc_amt.reset_index()
plt.subplot(122)
g1 = sns.barplot(x='isFraud', y='TransactionAmt', dodge=True, data=perc_amt)
g1.set_title('% Total Amount in Transaction Amt \n# 0: No Fraud | 1: Fraud #', fontsize=22)
g1.set_xlabel('Is Fraud?', fontsize=18)
g1.set_ylabel('Total Transaction Amount Scalar', fontsize=18)
# print(g1.patches)
for p in g1.patches:
#     print(p)
    height = p.get_height()
    g1.text(p.get_x() + p.get_width() / 2.,
            height + 3,
            '{:1.2f}%'.format(height/total_amt*100),
            ha="center", fontsize=15)
plt.show()

사기 거래 확률은 전체 거래 중 3.5%에 해당한다.   
거래 금액으로 보았을 때는 3.87% 정도의 금액이 사기에 해당한다.

#### Transaction Amount

##### Transaction Amount Quantiles

In [ ]:
df_train['TransactionAmt'] = df_train['TransactionAmt'].astype(float)
print('Transaction Amounts Quantiles: ')
print(df_trans['TransactionAmt'].quantile([.01, .025, .1, .25, .5, .75, .9, .975, .99]))

In [ ]:
df_id['id_11'].value_counts(dropna=False, normalize=True)*100

##### Transaction Amount Distribution Plots

In [ ]:
plt.figure(figsize=(16, 12))
plt.suptitle('Transaction Values Distribution', fontsize=22)

plt.subplot(221)
g = sns.distplot(df_trans.loc[df_trans.TransactionAmt <= 1000]['TransactionAmt'])
g.set_title('Transaction Amount Distribution <= 1000', fontsize=18)
g.set_xlabel("")
g.set_ylabel('Probability', fontsize=15)

plt.subplot(222)
g1 = sns.distplot(df_trans['TransactionAmt'])
g1.set_title('Transaction Amount (Log) Distribution', fontsize=18)
g1.set_xlabel('')
g1.set_ylabel('Probability', fontsize=15)

plt.figure(figsize=(16, 12))

plt.subplot(212)
g4 = plt.scatter(range(df_trans.loc[df_trans.isFraud == 0].shape[0]),
                 np.sort(df_trans.loc[df_trans.isFraud == 0]['TransactionAmt'].values),
                 label='NoFraud', alpha=.2)
g4 = plt.scatter(range(df_trans.loc[df_trans.isFraud == 1].shape[0]),
                 np.sort(df_trans.loc[df_trans.isFraud == 1]['TransactionAmt'].values),
                 label='Fraud', alpha=.2)
g4 = plt.title('ECDF \nFRAUD and NO FRAUD Transaction Amount Distribution', fontsize=18)
g4 = plt.xlabel('Index')
g4 = plt.ylabel('Amount Distribution', fontsize=15)
g4 = plt.legend()

plt.figure(figsize=(16, 12))

plt.subplot(321)
g = plt.scatter(range(df_trans.loc[df_trans.isFraud == 1].shape[0]),                
                        np.sort(df_trans.loc[df_trans.isFraud == 1]['TransactionAmt'].values),
                        label='isFraud', alpha=.4)
plt.title('FRAUD - Transaction Amount ECDF', fontsize=18)
plt.xlabel('Index')
plt.ylabel('Amount Distribution', fontsize=12)

plt.subplot(322)
g1 = plt.scatter(range(df_trans.loc[df_train.isFraud == 0].shape[0]),
                 np.sort(df_trans.loc[df_train.isFraud == 0]['TransactionAmt'].values),
                 label='NoFraud', alpha=.2)
g1 = plt.title('NO FRAUD - Transaction Amount ECDF', fontsize=18)
g1 = plt.xlabel('Index')
g1 = plt.ylabel('Amount Distribution', fontsize=15)

plt.suptitle('Individual ECDF Distribution', fontsize=22)
plt.show()

금액 200 언저리의 비교적 소액 거래에 대한 사기가 많은 것 같다.

#### Transaction Quantiles Per isFraud

In [ ]:
print(pd.concat([df_trans.loc[df_trans.isFraud==1]['TransactionAmt'].quantile([.01, .1, .25, .5, .75, .9, .99]).reset_index(),
                 df_trans.loc[df_trans.isFraud==0]['TransactionAmt'].quantile([.01, .1, .25, .5, .75, .9, .99]).reset_index()],
                axis=1, keys=['Fraud', 'No Fraud']))

> 사기거래와 비 사기거래의 백분위수는 거의 흡사한 것 같다.

##### Transaction Amount Outliers

In [ ]:
# Calculate outliers of Transaction Amount
calc_outliers(df_trans['TransactionAmt'])

이상치는 모두 표준편차 3배보다 높은 곳에 속해있다.   
전체의 약 1.71%가 이상치다.

#### Product Features

In [ ]:
tmp

In [ ]:
# ProductCD별 사기/not사기의 비율 측정 테이블
tmp = pd.crosstab(df_trans['ProductCD'], df_trans['isFraud'], normalize='index') * 100
tmp = tmp.reset_index()
tmp.rename(columns={0: 'NoFraud', 1: 'Fraud'}, inplace=True)

plt.figure(figsize=(14, 10))
plt.suptitle('ProductCD Distributions', fontsize=22)

plt.subplot(221)
g = sns.countplot(x='ProductCD', data=df_trans)

# 각 ProductCD별 카운트 그래프
g.set_title('ProductCD Distribution', fontsize=19)
g.set_xlabel('ProductCD Name', fontsize=17)
g.set_ylabel('Count', fontsize=17)
g.set_ylim(0, 500000)
for p in g.patches:
    height = p.get_height()
    g.text(p.get_x()+p.get_width() / 2.,
         height+3,
         '{:1.2f}'.format(height/total*100),
         ha="center", fontsize=14)

# ProductCD별 사기/not사기 비율 막대 그래프 및 전체 Product CD 중에서의 사기 비율(오른쪽 y축)
plt.subplot(222)
g1 = sns.countplot(x='ProductCD', hue='isFraud', data=df_trans)
plt.legend(title='Fraud', loc='best', labels=['No', 'Yes'])
gt = g1.twinx()
gt = sns.pointplot(x='ProductCD', y='Fraud', data=tmp, color='black', order=['W','H','C','S','R'], legend=False)
gt.set_ylabel("% of Fraud Transactions", fontsize=16)


g1.set_title("Product CD by Target(isFraud)", fontsize=19)
g1.set_xlabel("ProductCD Name", fontsize=17)
g1.set_ylabel("Count", fontsize=17)

plt.subplot(212)
g3 = sns.boxenplot(x='ProductCD', y='TransactionAmt', hue='isFraud',
                   data=df_trans.loc[df_trans.TransactionAmt <= 2000])
g3.set_title('Transaction Amount Distribuition by ProductCD and Target', fontsize=20)
g3.set_xlabel('ProductCD Name', fontsize=17)
g3.set_ylabel('Transaction Values', fontsize=17)

plt.subplots_adjust(hspace=0.6, top=0.85)
plt.show()

Product Code는 W의 수가 압도적으로 높은데 사기거래 비율이 가장 낮고, C의 사기 비율이 압도적으로 높다.

#### Card Feature

Card features are categorical. Let's see the distributions.

In [ ]:
# 카드 피쳐 요약
resume_table(df_trans[['card1', 'card2', 'card3', 'card4', 'card5', 'card6']])

카드1 외에는 모두 결측치가 포함되어 있다.

##### Numerical Card Features quantile(card1,2,3,5)

In [ ]:
print('Card Features Quantiles: ')
print(df_trans[['card1', 'card2', 'card3', 'card5']].quantile([.01, .025, .1, .25, .5, .75, .975, .99]))

card1과 card2가 비교적 분포가 넓은 편인 것 같다.
로그값을 취하면 괜찮은 수치가 나올 거 같다.

In [ ]:
np.log(df_trans['card1'])

In [ ]:
# 위에서 보는 것처럼 card3과 card5의 경우, 값의 범위가 굉장히 작은데 그 중에 값이 적은 것들이 굉장히 많다.
# 그러므로 카드3과 카드5는 각각 200개 미만, 300개 미만인 값들을 'Others'로 묶도록 한다.
df_trans.loc[df_trans.card3.isin(df_trans.card3.value_counts()[df_trans.card3.value_counts() < 200].index), 'card3'] = 'Others'
df_trans.loc[df_trans.card5.isin(df_trans.card5.value_counts()[df_trans.card5.value_counts() < 300].index), 'card5'] = 'Others'

##### Plot Card 1,2,3 distributions
- 카드 1과 2는 수치형 피쳐이기에 이것들의 분포를 시각화 해 보도록 하자.
- 카드 3은 낮은 빈도의 값들이 많아서 그것들을 Others로 묶도록 하자.
- 또한 카드 3에서 yaxis2에서의 사기 비율에 대해 %값으로 세팅해 보도록 한다.

In [ ]:
# card3 안의 카테고리별 사기/not사기 비율
tmp = pd.crosstab(df_trans.card3, df_trans.isFraud, normalize='index') * 100
tmp = tmp.reset_index()
tmp.rename(columns={0: 'NoFraud', 1:'Fraud'}, inplace=True)

# card5 안의 카테고리별 사기/not사기 비율
tmp2 = pd.crosstab(df_trans.card5, df_trans.isFraud, normalize='index') * 100
tmp2 = tmp2.reset_index()
tmp2.rename(columns={0:'NoFraud', 1:'Fraud'}, inplace=True)

plt.figure(figsize=(14, 22))

# card1의 목표값별 카테고리 카운트
plt.subplot(411)
g = sns.histplot(df_trans, x='card1', hue='isFraud', kde=True, palette='gnuplot2_r')
g.legend()
g.set_title('Card 1 Values Distribution by Target', fontsize=20)
g.set_xlabel('Card 1 Values', fontsize=18)
g.set_ylabel('Probability', fontsize=18)

# card2의 목표값별 카테고리 카운트
plt.subplot(412)
g = sns.histplot(df_trans, x='card2', hue='isFraud', kde=True, palette='gnuplot2_r')
g.legend()
g.set_title('Card 2 Values Distribution by Target', fontsize=20)
g.set_xlabel('Card 2 Values', fontsize=18)
g.set_ylabel('Probability', fontsize=18)

# card3의 목표값별 카테고리 카운트와 카테고리별 사기 비율
plt.subplot(413)
g2 = sns.countplot(x='card3', data=df_trans, order=list(tmp.card3.values))
g22 = g2.twinx() # Make 2 Y in one ax
gg2 = sns.pointplot(x='card3', y='Fraud', data=tmp, color='black', order=list(tmp.card3.values))
gg2.set_ylabel('% of Fraud Transactions', fontsize=16)
g2.set_title('Card 3 Values Distribution and % of Transaction Frauds', fontsize=20)
g2.set_xlabel('Card 3 Values', fontsize=18)
g2.set_ylabel('Count', fontsize=18)
for p in g2.patches:
    height = p.get_height()
    g2.text(p.get_x()+p.get_width()/2.,
          height+25,
          '{:1.2f}%'.format(height/total*100),
          ha='center')

plt.subplot(414)
g3 = sns.countplot(x='card5', data=df_trans, order=list(tmp2.card5.values), palette='rainbow')
g3t = g3.twinx() # # Make 2 Y in one ax
g3t = sns.pointplot(x='card5', y='Fraud', data=tmp2, color='black', order=list(tmp2.card5.values))
g3t.set_ylabel("% of Fraud Transactions", fontsize=16)
g3.set_title("Card 5 Values Distribution and % of Transaction Frauds", fontsize=20)
g3.set_xticklabels(g3.get_xticklabels(), rotation=90)
g3.set_xlabel('Card 5 Values', fontsize=18)
g3.set_ylabel('Count', fontsize=18)
for p in g3.patches:
    height = p.get_height()
    g3.text(p.get_x()+p.get_width()/2.,
          height+3,
          '{:1.2f}%'.format(height/total*100),
          ha='center', fontsize=11)

plt.subplots_adjust(hspace=0.6, top=0.85)

plt.show()

In [ ]:
print(df_trans.card1.value_counts(dropna=False)/df_trans.shape[0]*100, df_trans.card2.value_counts(dropna=False)/df_trans.shape[0]*100)

Card1과 Card2는 넓은 범위 안에서 여러 값들이 고르게 분포하면서 백분율이 높은 값들이 들쭉날쭉하게 있는 패턴이다.   

Card3은 150, 185 순으로 값이 크면서 이 둘이 대부분을 차지한다.   
그러나 150에서는 사기확률이 약 2%를 조금 넘지만, 185에서 사기가 12%를 넘는 게 인상적이다.   

Card5 또한 226, 224, 166 순으로 값이 크면서 세 값이 총합 약 70%를 차지한다.
그러나 이 역시 Card3처럼 제일 높은 값이 사기 확률까지 높지는 않은 반면에 137, 147 순으로 사기 확률이 매우 큰 것을 볼 수 있다.

##### Card4 - Categorical

In [ ]:
tmp = pd.crosstab(df_trans.card4, df_trans.isFraud, normalize='index') * 100
tmp = tmp.reset_index()
tmp.rename(columns={0: 'NoFraud', 1:'Fraud'}, inplace=True)

plt.figure(figsize=(14, 10))
plt.suptitle('Card 4 Distributions', fontsize=22)

plt.subplot(221)
g = sns.countplot(x='card4', data=df_trans, palette='rainbow')
g.set_title('Card4 Distribution', fontsize=19)
g.set_ylim(0, 420000)
g.set_xlabel('Card4 Category Names', fontsize=17)
g.set_ylabel('Count', fontsize=17)
for p in g.patches:
    height = p.get_height()
    g.text(p.get_x()+p.get_width()/2.,
           height+3,
           '{:1.2f}%'.format(height/total*100),
           ha="center", fontsize=14)

plt.subplot(222)
g1 = sns.countplot(x='card4', hue='isFraud', data=df_trans)
plt.legend(title='Fraud', loc='best', labels=['No', 'Yes'])
gt = g1.twinx()
gt = sns.pointplot(x='card4', y='Fraud', data=tmp, color='black', legend=False, order=['discover', 'mastercard','visa','american express'])
gt.set_ylabel("% of Fraud Transactions", fontsize=16)
g1.set_title("Card4 by Target(isFraud)", fontsize=19)
g1.set_xlabel("Card4 Category Names", fontsize=17)
g1.set_ylabel("Count", fontsize=17)

plt.subplot(212)
g3 = sns.boxenplot(x='card4', y='TransactionAmt', hue='isFraud', data=df_trans.loc[df_trans.TransactionAmt <= 2000])
g3.set_title("Card 4 Distribution by ProductCD and Target", fontsize=20)
g3.set_xlabel("Card 4 Category Names", fontsize=17)
g3.set_ylabel("Transaction Values", fontsize=17)

plt.subplots_adjust(hspace=0.6, top=0.85)

plt.show()

전체 거래의 거의 97%를 비자(65%)와 마스터(32%)가 차지한다.
discover 카드가 전체 거래 비율이 낮은데에 비해 사기 비율이 다른 카드의 2배 정도이다.

##### Card 6 - Categorical

In [ ]:
# 카드 타입별(선불,신용,직불,직불or신용) 사기 비율
tmp = pd.crosstab(df_trans.card6, df_trans.isFraud, normalize='index') * 100
tmp = tmp.reset_index()
tmp.rename(columns={0: 'NoFraud', 1: 'Fraud'}, inplace=True)

plt.figure(figsize=(14, 10))
plt.suptitle('Card 6 Distributions', fontsize=22)

plt.subplot(221)
g = sns.countplot(x='card6', data=df_trans, order=list(tmp.card6.values))
g.set_title('Card6 Distribution', fontsize=19)
g.set_ylim(0, 480000)
g.set_xlabel('Card 6 Category Names', fontsize=17)
g.set_ylabel('Count', fontsize=17)
for p in g.patches:
    height = p.get_height()
    g.text(p.get_x()+p.get_width()/2.,
         height+3,
         '{:1.2f}%'.format(height/total*100),
         ha="center", fontsize=14)
plt.subplot(222)
g1 = sns.countplot(x='card6', hue='isFraud', data=df_trans, order=list(tmp.card6.values))
plt.legend(title='Fraud', loc='best', labels=['No', 'Yes'])
gt = g1.twinx()
gt = sns.pointplot(x='card6', y='Fraud', data=tmp, order=list(tmp.card6.values),
                   color='black', legend=False)
gt.set_ylim(0, 20)
gt.set_ylabel('% of Fraud Transactions', fontsize=16)
g1.set_title('Card6 by Target(isFraud)', fontsize=19)
g1.set_xlabel('Card 6 Category Names', fontsize=17)
g1.set_ylabel('Count', fontsize=17)

plt.subplot(212)
g3 = sns.boxenplot(x='card6', y='TransactionAmt', hue='isFraud', order=list(tmp.card6.values), data=df_trans.loc[df_trans.TransactionAmt <= 2000])
g3.set_title('Card 6 Distribution by ProductCD and Target', fontsize=20)
g3.set_xlabel('Card 6 Category Names', fontsize=17)
g3.set_ylabel('Transaction Values', fontsize=17)

plt.subplots_adjust(hspace=0.6, top=0.85)

plt.show()

거래 전체 카드 종류로는 직불카드가 약 75%로 압도적이다.
선불카드나 직불카드에 비해 압도적으로 신용카드가 사기 비율이 높다.
선불카드는 특성상 돈을 먼저 내는 것이다 보니 사기가 있을 수가 없는 것 같다.(?)

#### M1~M9

In [ ]:
for col in ['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']:
    df_trans[col] = df_trans[col].fillna('Miss')

def ploting_dist_ratio(df, col, lim=2000):
    """count plot과 box plot을 그리는 함수"""
    tmp = pd.crosstab(df[col], df.isFraud, normalize='index') * 100
    tmp = tmp.reset_index()
    tmp.rename(columns={0: 'NoFraud', 1: 'Fraud'}, inplace=True)

    plt.figure(figsize=(20, 5))
    plt.suptitle(f'{col} Distributions ', fontsize=22)

    plt.subplot(121)
    g = sns.countplot(x=col, data=df_trans, order=list(tmp[col].values))
    g.set_title(f'{col} Distribution\nCount and % Fraud by each Category', fontsize=18)
    g.set_ylim(0, 400000)
    gt = g.twinx()
    gt = sns.pointplot(x=col, y='Fraud', data=tmp, order=list(tmp[col].values), color='black', legend=False)
    gt.set_ylim(0, 20)
    gt.set_ylabel('% of Fraud Transactions', fontsize=16)
    g.set_xlabel(f'{col} Category Names', fontsize=16)
    g.set_ylabel('Count', fontsize=17)
    for p in gt.patches:
        height = p.get_height()
        gt.text(p.get_x()+p.get_width()/2.,
            height+3,
            '{:1.2f}%'.format(height/total*100),
            ha='center', fontsize=14)
    perc_amt = (df_trans.groupby(['isFraud', col])['TransactionAmt'].sum() / total_amt * 100).unstack('isFraud')
    perc_amt = perc_amt.reset_index()
    perc_amt.rename(columns={0: 'NoFraud', 1: 'Fraud'}, inplace=True)

    plt.subplot(122)
    g1 = sns.boxplot(x=col, y='TransactionAmt', hue='isFraud', data=df.loc[df.TransactionAmt <= lim], order=list(tmp[col].values))
    g1t = g1.twinx()
    g1t = sns.pointplot(x=col, y='Fraud', data=perc_amt, order=list(tmp[col].values), color='black', legend=False)
    g1t.set_ylim(0, 5)
    g1t.set_ylabel('%Fraud Total Amount', fontsize=16)
    g1.set_title(f'{col} by Transaction dist', fontsize=18)
    g1.set_xlabel(f'{col} Category Names', fontsize=16)
    g1.set_ylabel('Transaction Amount(U$)', fontsize=16)

    plt.subplots_adjust(hspace=.4, wspace=.35, top=.8)

    plt.show()

In [ ]:
for col in ['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']:
    ploting_dist_ratio(df_trans, col, lim=2500)

#### Addr1 and Addr2

In [ ]:
print('Card Features Quantiles: ')
print(df_trans[['addr1', 'addr2']].quantile([.01, .025, .1, .25, .5, .75, .9, .975, .99]))

In [ ]:
# Replace Addr1 as 'Others' where the value counts less than 5000
df_trans.loc[df_trans.addr1.isin(df_trans.addr1.value_counts()[df_trans.addr1.value_counts() <= 5000].index), 'addr1'] = 'Others'
# Addr2에 관해서 값이 50 이하인 값에 대해 Others로 값을 대체한다.
df_trans.loc[df_trans.addr2.isin(df_trans.addr2.value_counts()[df_trans.addr2.value_counts() <= 50].index), 'addr2'] = 'Others'

##### Addr1 Distribution

In [ ]:
def ploting_cnt_amt(df, col, lim=2000):
    tmp = pd.crosstab(df[col], df['isFraud'], normalize='index') * 100
    tmp = tmp.reset_index()
    tmp.rename(columns={0: 'NoFraud', 1: 'Fraud'}, inplace=True)

    plt.figure(figsize=(16, 14))
    plt.suptitle(f'{col} Distributions ', fontsize=24)

    plt.subplot(211)
    g = sns.countplot(x=col, data=df, order=list(tmp[col].values))
    gt = g.twinx()
    gt = sns.pointplot(x=col, y='Fraud', data=tmp, order=list(tmp[col].values), 
                     color='black', legend=False)
    gt.set_ylim(0, tmp['Fraud'].max()*1.1)
    gt.set_ylabel('%Fraud Transactions', fontsize=16)
    g.set_title(f'Most Frequent {col} values and % Fraud Transactions', 
              fontsize=20)
    g.set_xlabel(f'{col} Category Names', fontsize=16)
    g.set_ylabel('Count', fontsize=17)
    g.set_xticklabels(g.get_xticklabels(), rotation=45)
    sizes = []
    for p in g.patches:
        height = p.get_height()
        sizes.append(height)
        g.text(p.get_x()+p.get_width()/2.,
               height+3,
               '{:1.2f}%'.format(height/total*100),
               ha="center", fontsize=12)
    g.set_ylim(0, max(sizes) * 1.15)

    ##############################################################################
    perc_amt = (df.groupby(['isFraud', col])['TransactionAmt'].sum() \
                / df.groupby([col])['TransactionAmt'].sum() * 100).unstack('isFraud')
    perc_amt = perc_amt.reset_index()
    perc_amt.rename(columns={0: 'NoFraud', 1: 'Fraud'}, inplace=True)
    amt = df.groupby([col])['TransactionAmt'].sum().reset_index()
    perc_amt = perc_amt.fillna(0)
    plt.subplot(212)
    g1 = sns.barplot(x=col, y='TransactionAmt', data=amt, order=list(tmp[col].values))
    g1t = g1.twinx()
    g1t = sns.pointplot(x=col, y='Fraud', data=perc_amt,
                        order=list(tmp[col].values),
                        color='black', legend=False)
    g1t.set_ylim(0, perc_amt['Fraud'].max()*1.1)
    g1t.set_ylabel("%Fraud Total Amount", fontsize=16)
    g.set_xticklabels(g.get_xticklabels(), rotation=45)
    g1.set_title(f"{col} by Transactions Total + %of total and %Fraud Transactions", fontsize=20)
    g1.set_xlabel(f"{col} Category Names", fontsize=16)
    g1.set_ylabel("Transaction Total Amount(U$)", fontsize=16)
    g1.set_xticklabels(g.get_xticklabels(), rotation=45)

    for p in g1.patches:
        height = p.get_height()
        g1.text(p.get_x()+p.get_width()/2.,
                height+3,
                '{:1.2f}%'.format(height/total_amt*100),
                ha="center", fontsize=12)
    plt.subplots_adjust(hspace=.4, top=.9)
    plt.show()
ploting_cnt_amt(df_trans, 'addr1')

##### Addr2 Distribution

In [ ]:
ploting_cnt_amt(df_trans, 'addr2')

addr2의 65에서 고확률로 사기가 일어나고 있다. 무려 60%에 달한다.
addr2중에 87이 대부분(96%)을 차지하지만 사기 확률은 3% 정도인 것 같다.

##### P email Domain Distribution

> P 이메일 도메인을 기업에 따라 그룹지어보고 500개가 안 되는 도메인 기업은 Others로 묶어주자.

In [ ]:
df_trans.loc[df_trans.P_emaildomain.isin(['gmail.com', 'gmail']), 'P_emaildomain'] = 'Google'
df_trans.loc[df_trans.P_emaildomain.isin(['yahoo.com', 'yahoo.com.mx', 'yahoo.co.uk',
                                                            'yahoo.co.jp', 'yahoo.de', 'yahoo.fr',
                                                            'yahoo.es']), 'P_emaildomain'] = 'Yahoo Email'
df_trans.loc[df_trans.P_emaildomain.isin(['hotmail.com', 'outlook.com', 'msn.com', 'live.com.mx',
                                                            'hotmail.es', 'hotmail.co.uk', 'hotmail.de',
                                                            'outlook.es', 'live.com', 'live.fr', 'hotmail.fr']), 'P_emaildomain'] = 'Microsoft'
df_trans.loc[df_trans.P_emaildomain.isin(df_trans.P_emaildomain\
                                                           .value_counts()[df_trans.P_emaildomain.value_counts() <= 500].index
                                                           ), 'P_emaildomain'] = 'Others'
df_trans.P_emaildomain.fillna('NoInf', inplace=True)

##### P email 도메인 ploting

In [ ]:
ploting_cnt_amt(df_trans, 'P_emaildomain')

Gmail 유저가 제일 많은가보다. 구글이 대부분을 차지하고 있다.
mail.com이라는 수상한 도메인이 사기확률 및 사기 금액 모두 1등을 차지하였다.

###### R email domain distribution

> P 이메일 도메인과 마찬가지로 R 이메일 도메인을 기업에 따라 그룹지어보고 500개가 안 되는 도메인 기업은 Others로 묶어주자.

In [ ]:
df_trans.loc[df_trans.R_emaildomain.isin(['gmail.com', 'gmail']), 'R_emaildomain'] = 'Google'
df_trans.loc[df_trans.R_emaildomain.isin(['yahoo.com', 'yahoo.com.mx', 'yahoo.co.uk',
                                                            'yahoo.co.jp', 'yahoo.de', 'yahoo.fr',
                                                            'yahoo.es']), 'R_emaildomain'] = 'Yahoo Email'
df_trans.loc[df_trans.R_emaildomain.isin(['hotmail.com', 'outlook.com', 'msn.com', 'live.com.mx',
                                                            'hotmail.es', 'hotmail.co.uk', 'hotmail.de',
                                                            'outlook.es', 'live.com', 'live.fr', 'hotmail.fr']), 'R_emaildomain'] = 'Microsoft'
df_trans.loc[df_trans.R_emaildomain.isin(df_trans.R_emaildomain\
                                                           .value_counts()[df_trans.R_emaildomain.value_counts() <= 300].index
                                                           ), 'R_emaildomain'] = 'Others'
df_trans.R_emaildomain.fillna('NoInf', inplace=True)

In [ ]:
ploting_cnt_amt(df_trans, 'R_emaildomain')

#### C1-C14

마스킹된 C1에서 C14의 피쳐들에 대해 살펴보자.

In [ ]:
resume_table(df_trans[['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8',
                               'C9', 'C10', 'C11', 'C12', 'C13', 'C14']])

신기하게도 C 피쳐들은 결측치가 전혀 없다.

In [ ]:
df_trans[['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8',
                               'C9', 'C10', 'C11', 'C12', 'C13', 'C14']].describe()

In [ ]:
# C1 값 중에 400개 이하인 행에 대해서는 Others로 값을 대체한다.
df_trans.loc[df_trans.C1.isin(df_trans.C1.value_counts()[df_trans.C1.value_counts() <= 400 ].index), 'C1'] = "Others"

##### C1 Distribution

In [ ]:
ploting_cnt_amt(df_trans, 'C1')

신기하게도 C1이 0일 때 엄청난 사기 확률을 보이는데 1이 가장 낮으며, 그 위로 C1의 값이 올라갈수록 사기 확률이 올라가는 것을 볼 수 있다.

In [ ]:
# C2 값 중에 350개 이하인 행에 대해서는 Others로 값을 대체한다.
df_trans.loc[df_trans.C2.isin(df_trans.C2.value_counts()[df_trans.C2.value_counts() <= 350 ].index), 'C2'] = "Others"

In [ ]:
df_trans.C2.value_counts()

In [ ]:
ploting_cnt_amt(df_trans, 'C2')

10, 13, 15에서 사기 금액 비율이 높다.

#### TimeDelta Feature

맨 처음에 언급하였던 것처럼 TransactionDT는 타임델타를 이야기한다.   
여기에서는 2017년 12월 1일을 기준으로 하여 거래시각을 계산하도록 하겠다.

In [ ]:
import datetime

START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d') # 2017년 12월 1일의 타임스탬프 변환 값
df_trans['Date'] = df_trans.TransactionDT.apply(lambda x: (startdate + datetime.timedelta(seconds=x))) # 각 행에 Date열을 만들어 날짜 계산을 적용

df_trans['_Weekdays'] = df_trans.Date.dt.dayofweek # 요일
df_trans['_Hours'] = df_trans.Date.dt.hour # 시간
df_trans['_Days'] = df_trans.Date.dt.day # 일

In [ ]:
ploting_cnt_amt(df_trans, '_Days')

주로 월초,월말에 사기가 많이 일어나나보다.   
그 중 사기 금액은 월말에 일어나는 것이 금액 비율이 높다.

##### Ploting WeekDays Distributions

In [ ]:
ploting_cnt_amt(df_trans, '_Weekdays')

금토일이 사기 비율이 높고, 거래 금액은 월요일이 가장 적지만 사기 금액 비율은 월요일이 가장 높다.

##### Ploting Hours Distributions

In [ ]:
ploting_cnt_amt(df_trans, '_Hours')

오전시간대에 사기 금액도, 비율도 높다.

##### Transactions and Total Amount by each day

In [ ]:
# 컬러 리스트
color_op = ['#5527A0', '#BB93D7', '#834CF7', '#6C941E', '#93EAEA', '#7425FF', '#F2098A', '#7E87AC', 
            '#EBE36F', '#7FD394', '#49C35D', '#3058EE', '#44FDCF', '#A38F85', '#C4CEE0', '#B63A05', 
            '#4856BF', '#F0DB1B', '#9FDBD9', '#B123AC']

In [ ]:
dates_temp = df_trans.groupby(df_trans.Date.dt.date)['TransactionAmt'].count().reset_index()

trace = go.Scatter(x=dates_temp.Date, y=dates_temp.TransactionAmt, opacity=0.8, line=dict(color=color_op[7]), name='Total Transactions')

dates_temp_sum = df_trans.groupby(df_trans.Date.dt.date)['TransactionAmt'].sum().reset_index()

trace1 = go.Scatter(x=dates_temp_sum.Date, line=dict(color=color_op[1]), name='Total Amount', y=dates_temp_sum['TransactionAmt'], opacity=0.8, yaxis='y2')

layout = dict(
    title='Total Transactions and Fraud Informations by Date',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                          dict(count=1, label='1m', step='month', stepmode='backward'),
                          dict(count=3, label='3m', step='month', stepmode='backward'),
                          dict(count=6, label='6m', step='month', stepmode='backward'),
                          dict(step='all')
            ])
        ),
        rangeslider=dict(visible=True),
        type='date'),
    yaxis=dict(title='Total Transactions'),
    yaxis2=dict(overlaying='y', anchor='x', side='right', zeroline=False, showgrid=False, title='Total Transaction Amount')
)

fig = dict(data=[trace, trace1], layout=layout)
iplot(fig)

날짜별 사기 거래 추이   
웬지 모르게 2017년 12월 23일 전후, 2018년 3월 3일 전후가 다른 날들에 비해 매우 높다.

#### id features
identity 데이터 중에서 카테고리컬 피쳐에 해당하는 id_12에서 38까지를 살펴본다.

In [ ]:
df_id[['id_%d'%i for i in range(12, 25)]].describe(include='all')

In [ ]:
df_id[['id_%d'%i for i in range(25, 39)]].describe(include='all')

In [ ]:
df_train = df_trans.merge(df_id, how='left', left_index=True, right_index=True)

#### Ploting columns with few unique values

In [ ]:
def cat_feat_ploting(df, col):
    tmp = pd.crosstab(df[col], df['isFraud'], normalize='index') * 100
    tmp = tmp.reset_index()
    tmp.rename(columns={0: 'NoFraud', 1: 'Fraud'}, inplace=True)

    plt.figure(figsize=(14, 10))
    plt.suptitle(f'{col} Distributions ', fontsize=22)

    plt.subplot(221)
    g = sns.countplot(x=col, data=df, order=tmp[col].values)

    g.set_title(f"{col} Distribution", fontsize=19)
    g.set_xlabel(f"{col} Name", fontsize=17)
    g.set_ylabel("Count", fontsize=17)
    # g.set_ylim(0,500000)
    for p in g.patches:
        height = p.get_height()
        g.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(height/total*100),
                ha="center", fontsize=14) 

    plt.subplot(222)
    g1 = sns.countplot(x=col, hue='isFraud', data=df, order=tmp[col].values)
    plt.legend(title='Fraud', loc='best', labels=['No', 'Yes'])
    gt = g1.twinx()
    gt = sns.pointplot(x=col, y='Fraud', data=tmp, color='black', order=tmp[col].values, legend=False)
    gt.set_ylabel("% of Fraud Transactions", fontsize=16)

    g1.set_title(f"{col} by Target(isFraud)", fontsize=19)
    g1.set_xlabel(f"{col} Name", fontsize=17)
    g1.set_ylabel("Count", fontsize=17)

    plt.subplot(212)
    g3 = sns.boxenplot(x=col, y='TransactionAmt', hue='isFraud', 
                        data=df[df['TransactionAmt'] <= 2000], order=tmp[col].values )
    g3.set_title("Transaction Amount Distribuition by ProductCD and Target", fontsize=20)
    g3.set_xlabel("ProductCD Name", fontsize=17)
    g3.set_ylabel("Transaction Values", fontsize=17)

    plt.subplots_adjust(hspace = 0.4, top = 0.85)

    plt.show()

In [ ]:
df_trans.head()

In [ ]:
df_train.id_01.value_counts()

In [ ]:
for col in ['id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29']:
    df_train[col] = df_train[col].fillna('NaN')
    cat_feat_ploting(df_train, col)

In [ ]:
df_train.loc[df_train.id_30.str.contains('Windows', na=False), 'id_30'] = 'Windows'
df_train.loc[df_train.id_30.str.contains('iOS', na=False), 'id_30'] = 'iOS'
df_train.loc[df_train.id_30.str.contains('Mac', na=False), 'id_30'] = 'Mac OS'
df_train.loc[df_train.id_30.str.contains('Android', na=False), 'id_30'] = 'Android'
df_train.id_30.fillna('NAN', inplace=True)

In [ ]:
ploting_cnt_amt(df_train, 'id_30')

In [ ]:
df_train.loc[df_train['id_31'].str.contains('chrome', na=False), 'id_31'] = 'Chrome'
df_train.loc[df_train['id_31'].str.contains('firefox', na=False), 'id_31'] = 'Firefox'
df_train.loc[df_train['id_31'].str.contains('safari', na=False), 'id_31'] = 'Safari'
df_train.loc[df_train['id_31'].str.contains('edge', na=False), 'id_31'] = 'Edge'
df_train.loc[df_train['id_31'].str.contains('ie', na=False), 'id_31'] = 'IE'
df_train.loc[df_train['id_31'].str.contains('samsung', na=False), 'id_31'] = 'Samsung'
df_train.loc[df_train['id_31'].str.contains('opera', na=False), 'id_31'] = 'Opera'
df_train['id_31'].fillna("NAN", inplace=True)
df_train.loc[df_train.id_31.isin(df_train.id_31.value_counts()[df_train.id_31.value_counts() < 200].index), 'id_31'] = "Others"

In [ ]:
ploting_cnt_amt(df_train, 'id_31')

In [ ]:
df_train.loc[df_train['DeviceInfo'].str.contains('SM-', na=False),'DeviceInfo'] = 'SAMSUNG'
# df_train.loc[df_train['DeviceInfo'].str.contains('LG', na=False),'DeviceInfo'] = 'LG'

In [ ]:
# df_train.DeviceInfo.value_counts()
# df_train[df_train['DeviceInfo'].str.contains('LG-', na=False)]['DeviceInfo']
df_train['DeviceInfo'].value_counts()